In [1]:
import streamlit as st
import base64
import os
from ultralytics import YOLO
import pandas as pd
import pickle

In [4]:
path_upload = 'uploads'
picture_name=os.listdir(path_upload)[0]
picture_name

'dent1.jpg'

In [5]:
full_path= os.path.join(path_upload,picture_name)
# we import model
yolo_model = YOLO("Models/yolo_model.pt")

In [6]:
results = yolo_model.predict(full_path,conf=0.25)

# we save the image
path_output = os.path.join(path_upload, 'pred_'+picture_name)
results[0].save(filename=path_output)


image 1/1 C:\Users\daniq\PycharmProjects\project-danielteresa-API\uploads\dent1.jpg: 480x640 7 met_scratchs, 1 mis_lamp, 1 met_dent_minor, 1003.9ms
Speed: 0.0ms preprocess, 1003.9ms inference, 70.7ms postprocess per image at shape (1, 3, 480, 640)


'uploads\\pred_dent1.jpg'

In [8]:
with open('./Models/cost_model.pkl', 'rb') as file:
    cost_model = pickle.load(file)

In [9]:
buttom_workshop='Sandyford McCann Motor'

# Display the selected workshop
if buttom_workshop != "Select among available workshops":
    # workshop selection
    workshops={
    'Smithfield Autotech':'Low',
    'Sandyford McCann Motor':'Medium',
    'Mobile Mechanic':'High'
    }

    selected_workshop_qual = workshops[buttom_workshop]

    # we predict the cost of the claims
    case = pd.DataFrame({
    'const': [1],
    'brand_Volkswagen': [0],
    'model_Corolla': [0],
    'model_Golf': [0],
    'model_Polo': [0],
    'model_Tiguan': [0],
    'model_Yaris': [1],
    'veh_age_range_Newer': [1],
    'veh_age_range_Old': [0],
    'workshop_quality_Low': [0],
    'workshop_quality_Medium': [0],
    'counties_group2': [0],
    'counties_group3': [0],
    'damage_type_met_dent_medium': [0],
    'damage_type_met_dent_minor': [0],
    'damage_type_met_dent_severe': [0],
    'damage_type_met_tear': [0],
    'damage_type_mis_lamp': [0],
    'damage_type_mis_lost': [0],
    'damage_type_mis_punct': [0]
    })

    # Adjust case DataFrame based on selected_workshop_qual
    if selected_workshop_qual == 'High':
        case['workshop_quality_Low'] = [0]
        case['workshop_quality_Medium'] = [0]
    elif selected_workshop_qual == 'Medium':
        case['workshop_quality_Low'] = [0]
        case['workshop_quality_Medium'] = [1]
    elif selected_workshop_qual == 'Low':
        case['workshop_quality_Low'] = [1]
        case['workshop_quality_Medium'] = [0]

    damage_types = [results[0].names[int(cls)] for cls in results[0].boxes.cpu().numpy().cls]

In [15]:
  # Function to generate the complete_case DataFrame
def generate_complete_case(case, damage_types):
    complete_case = pd.DataFrame()

    for damage_type in damage_types:
        temp_case = case.copy()

        # Set the corresponding damage type to 1
        damage_column = f'damage_type_{damage_type}'
        if damage_column in temp_case.columns:
            temp_case[damage_column] = [1]

        # Append this row to the complete_case DataFrame
        complete_case = pd.concat([complete_case, temp_case], ignore_index=True)

    return complete_case


    # Generate the complete_case DataFrame
complete_case = generate_complete_case(case, damage_types)

In [16]:
complete_case

,const,brand_Volkswagen,model_Corolla,model_Golf,model_Polo,model_Tiguan,model_Yaris,veh_age_range_Newer,veh_age_range_Old,workshop_quality_Low,workshop_quality_Medium,counties_group2,counties_group3,damage_type_met_dent_medium,damage_type_met_dent_minor,damage_type_met_dent_severe,damage_type_met_tear,damage_type_mis_lamp,damage_type_mis_lost,damage_type_mis_punct
0,1,0,0,0,0,0,1,1,0,0,1,0,0,0,1,0,0,0,0,0
1,1,0,0,0,0,0,1,1,0,0,1,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,1,1,0,0,1,0,0,0,0,0,0,0,0,0
3,1,0,0,0,0,0,1,1,0,0,1,0,0,0,0,0,0,1,0,0
4,1,0,0,0,0,0,1,1,0,0,1,0,0,0,0,0,0,0,0,0
5,1,0,0,0,0,0,1,1,0,0,1,0,0,0,0,0,0,0,0,0
6,1,0,0,0,0,0,1,1,0,0,1,0,0,0,0,0,0,0,0,0
7,1,0,0,0,0,0,1,1,0,0,1,0,0,0,0,0,0,0,0,0
8,1,0,0,0,0,0,1,1,0,0,1,0,0,0,0,0,0,0,0,0


In [17]:

    # dataframe with the predictions
cost_predictions= pd.DataFrame({'Damages': damage_types,
    'Cost Estimates': cost_model.predict(complete_case).round(0)
    })

    # Calculate the total cost as the sum of all 'Cost Estimates'
total_cost = cost_predictions['Cost Estimates'].sum()

In [18]:
cost_predictions

,Damages,Cost Estimates
0,met_dent_minor,138.0
1,met_scratch,281.0
2,met_scratch,281.0
3,mis_lamp,191.0
4,met_scratch,281.0
5,met_scratch,281.0
6,met_scratch,281.0
7,met_scratch,281.0
8,met_scratch,281.0


In [19]:
rename_damage_type = {
        'mis_lost': 'Lost Parts',
        'met_tear': 'Torn',
        'met_scratch': 'Paint Scratches',
        'glass_crack': 'Broken Glass',
        'mis_punct': 'Puncture',
        'mis_lamp': 'Broken Lamp',
        'met_dent_minor': 'Minor Dent',
        'met_dent_medium': 'Medium Dent',
        'met_dent_severe': 'Severe Dent'
    }

cost_predictions["Damages"] = cost_predictions["Damages"].map(rename_damage_type)

    # Sort the "Damages" column alphabetically
cost_predictions = cost_predictions.sort_values("Damages").reset_index(drop=True)

    # Add a sequential number to each group of damage types
cost_predictions["count"] = cost_predictions.groupby("Damages").cumcount() + 1

    # Combine the original damage type with the sequential number
cost_predictions["Damages"] = cost_predictions["Damages"].astype(str) + ' ' + cost_predictions["count"].astype(str)

    # we define the length of the dataset
n=len(cost_predictions["Damages"])

In [20]:
cost_predictions

,Damages,Cost Estimates,count
0,Broken Lamp 1,191.0,1
1,Minor Dent 1,138.0,1
2,Paint Scratches 1,281.0,1
3,Paint Scratches 2,281.0,2
4,Paint Scratches 3,281.0,3
5,Paint Scratches 4,281.0,4
6,Paint Scratches 5,281.0,5
7,Paint Scratches 6,281.0,6
8,Paint Scratches 7,281.0,7


In [22]:
if n>0: # damage 1
    damage = cost_predictions["Damages"][0]
    damage_cost = cost_predictions["Cost Estimates"][0]
    formatted_damage_cost = f"{damage_cost:,.0f}"

In [25]:
if n>4: # damage 5
    damage = cost_predictions["Damages"][4]
    damage_cost = cost_predictions["Cost Estimates"][4]
    formatted_damage_cost = f"{damage_cost:,.0f}"

In [26]:
damage

'Paint Scratches 3'